In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
df = pd.read_csv('ann_cyc_data.csv', low_memory = False)

In [3]:
# Convert started_at and ended_at columns to datetime type

df['started_at'] = pd.to_datetime(df['started_at'])

df['ended_at'] = pd.to_datetime(df['ended_at'])

In [4]:
# Find and eliminate duplicate entries; the only ones were some strange copies of rides from one day
# in November that received alternate start dates in December

dupid = df.groupby('ride_id', as_index = False)['started_at'].count()

dupid = dupid[dupid['started_at']>1]

df = df.loc[~((df['ride_id'].isin(dupid['ride_id'])) & (df['started_at'].dt.month == 12))]

df = df.reset_index()

In [5]:
# Extract started_at and ended_at columns into separate time and date information

df['start_time'] = df['started_at'].dt.time

df['end_time'] = df['ended_at'].dt.time

df['day'] = df['started_at'].dt.weekday

df['day_name'] = df['started_at'].dt.day_name()

df['dur'] = df.apply(lambda x: x['ended_at'] - x['started_at'], axis = 1)
df['dur'] = df['dur'].dt.total_seconds()

df['month_name'] = df['started_at'].dt.month_name()

df['month'] = df['started_at'].dt.month

df['year'] = df['started_at'].dt.year

df['start_date'] = df['started_at'].dt.date

In [6]:
# Remove any rides less than a minute (this includes some incorrect records which show a negative ride duration)

df = df[df['dur'] > 60]

In [7]:
# Remove rides which started at HQ QR (these were for testing purposes)

df = df[df['start_station_name'] != 'HQ QR']

In [8]:
df = df.sort_values(by = 'start_date')

In [9]:
df.to_csv('ann_cyc_data_V2.csv')
df.to_pickle('ann_cyc_data_V2.pkl')